#   _Key Derivation Functions_ - Funkcje wyprowadzania klucza
## Jakub Dyrcz, Łukasz Kollbek
###   Podstawowe informacje
1.  Funkcje KDF to algorytmy kryptograficzne zwracające sekret lub wiele sekretów z tajnej wartości.    
        Np. Stworzenie klucza prywatnego (jeden sekret) z hasła użytkownika (tajna wartość).
1.  Metody te powstały ponieważ hasła użytkowników są przewidywalne i krótkie (niska entropia). Pojawia się potrzeba stworzenia mocnego klucza.  
Jednak nikt nie będzie pamiętał 64 znakowego hasła. Dlatego z pomocą przychodzi `key-stretching`.  
Proces w którym tworzony jest bezpieczniejszy i (teoretycznie) odporniejszy na ataki siłowe nowy klucz.  
`Key-streching` jest jedną z kluczowych technik zwiększania bezpieczeństwa klucza. Do key-streching'u można zaliczyć proste funkcje hashujące jak SHA-*, MD5 (nie używać!) lub szyfry blokowe.
1. Funkcje wyprowadzania klucza powinny być `deterministyczne`. Oznacza to, że dla tych samych danych wejściowych otrzymamy zawsze ten sam skrót niezależnie ile razy funkcja zostanie wywołana.
1. Key Derivation Functions mogą być używane nie tylko do zapisywania haseł w postaci skrótów w bazie danych. Można je wykorzystywać między innymi do symetrycznego szyfrowania dysków, kryptografi asymetrycznej oraz niektóre odnogi algorytmów znajdują swoje zastosowanie w technologiach `blockchain`.
```
                                           _          _  ______  _____                               _       
 _ __   __ _ ___ _____      _____  _ __ __| |        | |/ /  _ \|  ___|       ___  ___  ___ _ __ ___| |_ ___  
| '_ \ / _` / __/ __\ \ /\ / / _ \| '__/ _` | =====\ | ' /| | | | |_   ====\ / __|/ _ \/ __| '__/ _ \ __/ __|
| |_) | (_| \__ \__ \\ V  V / (_) | | | (_| | =====/ | . \| |_| |  _|  ====/ \__ \  __/ (__| | |  __/ |_\__ \
| .__/ \__,_|___/___/ \_/\_/ \___/|_|  \__,_|        |_|\_\____/|_|          |___/\___|\___|_|  \___|\__|___/
|_|                                                     
```

---

### Funkcje KDF to nie tylko funkcje haszujące! Jednak mają wspólną część
1. Tak samo jak funkcje haszujące szyfrują dane deterministycznie. (To samo wejście - to samo wyjście)
1. Na wyjściu jest zawsze ciąg o takiej samej długości. Bez znaczenia jak długie lub krótkie były dane wejściowe.
1. Działa tylko w jedną stronę. Nie możliwe jest odzyskanie wejścia na podstawie wyjścia.


### Zrozumienie od podstaw

Najprostszą funkcję wyprowadzenia klucza można stworzyć wykorzystując dowolny algorytm haszujący. Dla przykładu użyty zostanie algorytm SHA256.  

function(`password`, `sha256`) --> `key`

In [9]:
from hashlib import sha256

hash = sha256(b"password")

print(hash.digest().hex())

5e884898da28047151d0e56f8dc6292773603d0d6aabbdd62a11ef721d1542d8


Należy pamiętać, że powyższe rozwiązanie nie jest bezpieczne. Proste hasze są podatne na ataki słownikowe (np. tęczowe tablice (Rainbow table)). Jak zatem zwiększyć bezpieczeństwo?

Pierwszym krokiem jest dodanie `soli` (ang. `salt`).  
`Salt` jest wartością przechowywaną razem z uzyskanym kluczem. Używana, żeby utrudnić atak siłowy (np. Rainbow Table). Sprawia, że nowy hasz jest bardziej skomplikowany i zmniejsza szansę pojawienia się go we wcześniej obliczonych tęczowych tablicach.

function_with_salt(`salt`, `password`, `sha256`) --> `key`  

In [10]:
password = b"password"
salt = b"salt"
hash = sha256(salt+password)

print(hash.digest().hex())

13601bda4ea78e55a07b98866d2be6be0744e3866f13c00c811cab608a28f322


Funkcję wyżej można opisać jako `HKDF` (`HMAC-based key derivation function`) - najprostsza funkcja wyprowadzania klucza.  
Ciągle jest mniej bezpieczna niż aktualne KDF. Zaleca się korzystanie z PBKDF2, Bcrypt, Scrypt (ten projekt) oraz Argon2.



### Przykład
Wykorzystanie `HMAC` (Hash-based message authentication code) z solą, hasłem użytkownika oraz funkcją SHA256 do pozyskania klucza.
1. Sprawdź co się stanie jak zmienisz sól. Może to być bardzo mała zmiana. Porównaj parę wyników dla tego samego hasła. 
1. Czy klucze z nową solą znacznie różnią się od głównego? Jeśli tak to dlaczego. ([Lawinowość](https://en.wikipedia.org/wiki/Avalanche_effect))
1. Porównaj dwa klucze z tym samym hasłem w następujący sposób:  
    1. Dla samej funkcji haszującej `sha256(b"password")`
    1. Następnie stwórz dwie różne sole.
    1. Porównaj klucze dołączając sól po haśle `sha256(b"password+salt")`

In [11]:
from hmac import HMAC
from hashlib import sha256

salt = b'NaCl'
password = b'CryptoIsFun'

key = HMAC(salt, password, digestmod=sha256)

print(key.digest().hex())
# Porządany wynik: `bff0e9482b9412e77f6b401c421957921782ff2f385521498ce59357fdc6da72`

bff0e9482b9412e77f6b401c421957921782ff2f385521498ce59357fdc6da72


### Zasosowanie `soli`
Wielokrotne użycie soli dla tego samego hasła umożliwia pozyskanie wielu innych kluczy. Jest to często stosowane rozwiązanie.  
Dla przykładu: można posiadać w bazie dwóch użytkowników z takim samym hasłem. Zwykłe zahaszowanie hasła w obu przypadkach zwróci to samo.  
Jednak wygenerowanie dla każdego z nich unikalnej wartości (`salt`) i dołączenie do hasła, sprawia że otrzymane hasze są diametralnie różne. ([Lawinowość](https://en.wikipedia.org/wiki/Avalanche_effect))  
KDF pomimo znajomości klucza z `salt1` nie jest wstanie znaleźć klucza z `salt2`.

### Po podstawach

Wiedząc co rozróżnia funkcje haszujące od KDF oraz czym jest sól, zwrócona zostanie uwaga na to, co powinna zapewniać dobra funkcja wyprowadzania klucza.  

1. Memory Hardness - cecha funkcji zapewniająca, że wymaga dużej ilości pamięci do obliczeń lub wymaga znacznie większego nakładu czasowego, tak by maszyny wykonujące obliczenia równoległe nie były znacząco bardziej efektywne w obliczeniach od standardowych procesorów CPU.  
Algorytm określa się "memory-hard" gdy potrzeba wysokiej ilości pamięci do pojedynczego wykonania. 

1. Funkcja KDF, która ma być wysoce odporna na ataki siłowe lub słownikowe, powinna maksymalnie spowolnić możliwość obliczeń równoległych. Jednocześnie wykonując relatywnie szybkie obliczenie klucza bądź skrótu.

1. Żeby "opóźnić" generację kluczy można zwiększyć złożoność obliczeniową, złożoność pamięciową lub oba jednocześnie.

Więcej informacji: `Funkcje wyprowadzania klucza (KDF) wykorzystujące odwzorowanie logistyczne - Grzegorz Frejek`

---

# Wprowadzenie do `scrypt`

`Scrypt` jest funkcją wyprowadzania klucza opartą na haśle. Implementacja opiera się na funkcjach, które zapewniają dodatkową ochronę (memory hardness).

Scrypt posiada więcej parametrów niż stworzony przez nas wcześniej przykład. 
Parametry scrypt są następujące:
```bash
    `P`      -   Passphrase
    `S`      -   Salt
    `N`      -   CPU/Memory cost (>1; potęga 2; < 2^(128 * r / 8))
    `r`      -   Block size
    `p`      -   Parallelization parametr (>0; <= ((2^32-1) * 32) / (128 * r)
    `dkLen`  -   Desired key length (oczekiwana długość klucza w bajtach; >0; <= (2^32 - 1) * 32)
```

Najpierw należy zaznajomić się z poszczególnymi funkcjami algorytmu. A potem działanie algorytmu można przedstawić w 3 krokach.

### Salsa20/8 - wariant [Salsa20](https://en.wikipedia.org/wiki/Salsa20)
Autor: Daniel J. Bernstein

`Salsa20/8` oraz `Salsa20/12` są wariantami z `Salsa20`. Nie zostały stworzone, żeby wyprzec oryginalną implementację. Wręcz przeciwnie. Mają za zadanie dopełniać i działać lepiej.  
Salsa20/8 nie może być uznana za funkcję haszującą ponieważ nie jest odporna na kolizje.

Aktualnie nie jest znana skuteczna metoda kryptoanalizy pełnej 20-rundowej Salsy (na 2015 rok). Wykryto wersje ataku na wersje z mniejszą liczbą rund. Szybszą kuzynką Salsy20 jest `ChaCha20` - miesza lepiej bity ale nie została przebadana jeszcze tak dobrze jak Salsa20.

Algorytm wykonuje operacje `XOR` na ciągach znaków w określonej liczbie rund. Salsa20 wykonuje 20 rund. Salsa20/8 i Salsa20/12 odpowiednio po 8 i 12 rund.

<!-- Można wstawić jakiś ładny opis. Troszkę bardziej rozbudowany. Co to robi czy coś. Chociaż kod mówi sam za siebie. -->
Implementacja Salsy20 w pythonie znajduje się poniżej.
<!-- Ja to bym wywalił wszystko do pythona. Potencjalne funkcje są w pliku salsa20.py albo salsa20_raw.py -->

Implementacja jest wzorowana na [link](https://github.com/Daeinar/salsa20).


Najistotniejszym elementem jest funkcja Quarter-Round. Następuje w niej xorowanie odpowiednio najpierw czterech kolumn. Potem transponujemy całą tablicę i przystępujemy do xorowania czterech wierszy. 

UWAGA: Kolejność operacji w funkcji QR() ma znaczenie. Muszą one odbywać się sekwencyjnie po sobie.

In [12]:
'''
Funkcja Ćwierć-Rundy
Przesunięcie cykliczne w lewo. Ważna kolejność!
Operacja add-rotate-XOR
'''
def QR(a, b, c, d):
    b = b ^ rot_left_32((a + d) & 0xffffffff, 7)
    c = c ^ rot_left_32((b + a) & 0xffffffff, 9)
    d = d ^ rot_left_32((c + b) & 0xffffffff, 13)
    a = a ^ rot_left_32((d + c) & 0xffffffff, 18)

'''
Funkcja pomocnicza - przesunięcie cykliczne w lewo
'''
def rot_left_32(a, b):
    return ((( a << b ) & 0xffffffff) | (a >> (32 - b)))

'''
Funkcja pomocnicza - transpozycja macierzy
'''
def transpose(state):
    state =[state[0],state[4],state[8],state[12],
            state[1],state[5],state[9],state[13],
            state[2],state[6],state[10],state[14],
            state[3],state[7],state[11],state[15]]
    return state

Kilka funkcji pomocniczych

In [13]:
'''
Funkcja pomocnicza - XOR na macierzy 4x4
'''
def xor_block(b1, b2):
    return [b1[i]^b2[i] for i in range(16)]


'''
Zwróć macierze 4x4 dla wiadomości odpowiedniej długości. (Bez obsługi paddingu!)
'''
def message_to_blocks(message):
    message_blocks = []
    for b in range(int(len(message)/64)):
        message_blocks.append([littleendian(message[64*b + 4*i : 4*i + 4 + 64*b]) for i in range(16)])
    return message_blocks


Każda runda to wykonanie operacji QR() na kolumnach i wierszach jednocześnie wykonując odpowiednie przesunięcie.

In [14]:
'''
Pełna runda to:
4 Ćwierć-Rundy (QR) -> transpozycja -> 4 Ćwierć-Rundy (QR) -> transpozycja

My dla czytelności implementujemy półrundy:
4 Ćwierć-Rundy -> transpozycja
Dlatego muszą zostać wykonane "dwukrotnie" na każde wykonanie
'''
def round(state):
    QR(state[0],state[4],state[8], state[12])
    QR(state[1],state[5],state[9], state[13])
    QR(state[2],state[6],state[10], state[14])
    QR(state[3],state[7],state[11], state[15])

    new_state = transpose(state)
    return new_state


Stan początkowy tworzy się według schematu. Jest to tablica 4x4.  

`Key` - klucz  
`Ctr.` - licznik  
`Nounce` - number used once  
`Const` - napis "expand 32-byte k" jako stałe  

|<!-- -->|<!-- -->|<!-- -->|<!-- -->|
|---|---|---|---|
|"expa"|Key|Key|Key|
|Key|"nd 3"|Nonce|Nonce|
|Ctr.|Ctr.|"2-by"|Key|
|Key|Key|Key|"te k"|



In [15]:
''' 
key - musi być długości 16 lub 32 bajtów
nuo - nonce (number used once) - unikalna liczba dla każdej wiadomości 2x4bytes
msg - wiadomość
con - stały ciąg znaków o wartości "expand 32 byte k"
'''
def salsa20(key, nuo, ctr):
    # Zamienione bajty na liczby:
    #     ['expa'    , 'nd 3'    , '2 by'    , 'te k']
    con = [0x61707865, 0x3320646e, 0x79622d32, 0x6b206574]
    key = [littleendian(key[4*i:4*i+4]) for i in range(8)]
    nuo = [littleendian(nuo[4*i:4*i+4]) for i in range(2)]
    ctr = [littleendian(ctr[4*i:4*i+4]) for i in range(2)]

    # Stan pierwotny
    init_state = [
        con[0], key[0], key[1], key[2],
        key[3], con[1], nuo[0], nuo[1],
        ctr[0], ctr[1], con[2], key[4],
        key[5], key[6], key[7], con[3]
    ]
    
    # Ilość rund może zostać zmieniona, w Salsa20/8 odbywa się 8 rund
    new_state = round(init_state) # Pierwsza runda
    for _ in range(15): # pozostałe rundy  (nasz kod dla czytelności implementuje pół-rundy, stąd 15)
        new_state = round(new_state)

    # Na koniec dodajemy całą przekształconą tablicę 16 słów do oryginalnej
    state = [(new_state[i] + init_state[i]) & 0xffffffff for i in range(16)]

    # Zwracamy dodane tablice
    return state


'''
Funkcja pomocnicza - Zmiana kolejności bajtów na Little Endian
'''
def littleendian(bytes):
     # przykład: 0x4A3B2C1D ==> [1D, 2C, 3B, 4A]
    return bytes[0] ^ (bytes[1] << 8) ^ (bytes[2] << 16) ^ (bytes[3] << 24)

Obecność stałej redukuje w pamięci obszar danych kontrolowanych przez atakującego. Niszczy potencjalne regularności wynikające z licznika, klucza i nounce. Przez co nie tworzy 'słabych' bloków.

Dodatkowo użycie tylko operacji `add-rotate-xor` zapobiega potencjalnym atakom czasowym w rozwiązaniach software'owych.

W ostatniej lini zmieszana tablica jest dodawana słowo po słowie, żeby oryginalna tablica uzyskała wielkość 64-bitowych bloków.  Jest to bardzo ważne ponieważ dodanie do oryginalnej tablicy zmieszancyh wartości sprawia, że niemożliwe jest pozyskanie danych wejściowych. Dokładnie ta sama technika jest szeroko wykorzystywana w funkcjach haszujących.

Sprawdźmy poprawność.

In [16]:
print("Testowanie poprawności implementacji:")

# Wybierz klucz
key="10101110100001010000001001111010"
key=[int(i) for i in key]
assert len(key) == 32

# Podaj nonce
nonce = [0,1,3,3,7,6,9,4]
assert len(nonce) == 8

# Inicjowanie licznika
counter = [0,0,0,0,0,0,0,0]
assert len(counter) == 8

# Wiadomość (128 bitowa)
message = "10001110111110101110101011010000101010001101001011111111100111011100111011100110111010101101211110101010110100101001111110011101"
assert len(message) == 128

# Zamień wiadomość na bloki po 16 znaków.
# Dla wiadomości długości 128 każda komórka po 4 powinna mieć 2 bloki.
message = [int(i) for i in message]
message_blocks = message_to_blocks(message)
assert len(message_blocks) == 2

print('Test kodowania i dekodowania dla wiadomości długości 128 dla 2 bloków')
print('Oryginalne bloki wiadomości')
print(message_blocks[0])
print(message_blocks[1])
cipher_msg = []
for i in range(2):
    counter[0] += 1
    block = salsa20(key, nonce, counter)
    cipher_msg.append(xor_block(block, message_blocks[i]))

print('Zakodowane bloki')
print(cipher_msg[0])        
print(cipher_msg[1])      


# Reset licznika przed dekodowaniem
counter = [0]*8

origin_msg = []
for i in range(2):
    counter[0] += 1
    block = salsa20(key, nonce, counter)
    origin_msg.append(xor_block(block, cipher_msg[i]))

print('Dekodowane bloki')
print(origin_msg[0])
print(origin_msg[1])
print("Są równe?", origin_msg == message_blocks)

Testowanie poprawności implementacji:
Test kodowania i dekodowania dla wiadomości długości 128 dla 2 bloków
Oryginalne bloki wiadomości
[1, 65793, 16843009, 65537, 65793, 65537, 16777473, 0, 65537, 1, 16777473, 65536, 16843009, 16843009, 16777217, 16777473]
[257, 65793, 65793, 65792, 65793, 65537, 16777473, 16843010, 65537, 65537, 16777473, 65536, 16777217, 16843009, 16777217, 16777473]
Zakodowane bloki
[3269521611, 196867, 16974595, 65539, 33620737, 1715587293, 117834497, 135400462, 65539, 1, 4089731941, 65536, 16974081, 50529025, 16908291, 3611347945]
[3269521867, 196867, 197379, 65794, 33620737, 1715587293, 117834497, 152243468, 65541, 65537, 4089731941, 65536, 16908289, 50529025, 16908291, 3611347945]
Dekodowane bloki
[1, 65793, 16843009, 65537, 65793, 65537, 16777473, 0, 65537, 1, 16777473, 65536, 16843009, 16843009, 16777217, 16777473]
[257, 65793, 65793, 65792, 65793, 65537, 16777473, 16843010, 65537, 65537, 16777473, 65536, 16777217, 16843009, 16777217, 16777473]
Są równe? True

### Algorytm `scryptBlockMix`
Scrypt wykorzystuje `scryptBlockMix` jako funkcję haszującą.

**Oficjalna reprezentacja algorytmu `scryptBlockMix`:**
```bash
  Parameters:
    r       Block size parameter.

  Input:
      B[0] || B[1] || ... || B[2 * r - 1]
          Input octet string (of size 128 * r octets),
          treated as 2 * r 64-octet blocks,
          where each element in B is a 64-octet block.

  Output:
      B[0] || B[1] || ... || B[2 * r - 1]
        Output octet string.

  Steps:

    1. X = B[2 * r - 1]

    2. for i = 0 to 2 * r - 1 do
        T = X xor B[i]
        X = Salsa (T)
        Y[i] = X
      end for

    3. B = (Y[0], Y[2], ..., Y[2 * r - 2], Y[1],
              Y[3], ..., Y[2 * r - 1])
```

Funkcja `scryptBlockMix` korzysta z parametru `r` podawanego przy wywołaniu `scrypt`

Jako wejście do funkcji podajemy ciąg bajtów o rozmiarze 128 * `r`, który wewnętrznie jest traktowany jako 2 * `r` 64-bajtowych bloków. Każdy element w `B` jest 64-bajtowym blokiem.

Kroki:
1. Rozkładamy blok wejściowy na 2 * `r` 64-bajtowych bloków.
1. Jako `X` wybieramy ostatni z nich
1. Wykonujemy "ilość bloków" razy poniższe instrukcje:
    1. Do zmiennej `T` zapisujemy wynik XOR(`X`, `aktualny_blok`)
    1. Nadpisujemy `X` przez wynik Salsa20/8(`T`)
    1. Dopisujemy wynikowe `X` do tablicy wyjściowej `Y`
1. Po wykonaniu pętli zwracamy tablicę `Y`

### Algorytm `scryptROMix`

Algorytm `scryptROMix` jest identyczny jak algorytm [`ROMix`](https://www.tarsnap.com/scrypt/scrypt.pdf) z tą różnicą, że wykorzystuje `scryptBlockMix` jako funkcję haszującą `H`.

To właśnie funkcja scryptROMix zapewnia własność `Memory Hardness` algorytmowi `scrypt`, ze względu na ogromną ilość pamięci jaką zajmuje w pamięci tablica `V`, która jest inna dla każdego wywołania.

Funkcja Integerify() zamienia podaną liczbę całkowitą zapisaną Big Endian na zapis Little Endian.

**Oficjalna reprezentacja algorytmu `scryptROMix`:**
```bash
Input:
        r       Block size parameter.
        B       Input octet vector of length 128 * r octets.
        N       CPU/Memory cost parameter, must be larger than 1,
                a power of 2, and less than 2^(128 * r / 8).

Output:
        B      Output octet vector of length 128 * r octets.

Steps:

    1. X = B

    2. for i = 0 to N - 1 do
        V[i] = X
        X = scryptBlockMix (X)
    end for

    3. for i = 0 to N - 1 do
        j = Integerify (X) mod N
                where Integerify (B[0] ... B[2 * r - 1]) is defined
                as the result of interpreting B[2 * r - 1] as a
                little-endian integer.
        T = X xor V[j]
        X = scryptBlockMix (T)
    end for

    4. B = X
```

Jako wejście do funkcji `scryptROMix` podajemy rozmiar bloku `r`, ciąg bajtów o rozmiarze 128 * `r` bajtów `B`, parametr kosztu Procesora/Pamięci `N`, który musi być większy od 1, potęgą 2 i mniejszy od 2^(128 * r / 8).

Kroki:
1. Jako wartość `X` przypisujemy oryginalny blok wejściowy `B`
1. Wykonujemy `N` razy poniższe instrukcje:
    1. Do tablicy `V` dopisujemy wartość `X`
    1. Nadpisujemy `X` przez wynik scryptBlockMix(`X`)
1. Po wykonaniu powyższej pętli wykonujemy `N` razy poniższe instrukcje:
    1. Nadpisujemy `j` przez wynik Integerify(`X`) modulo `N`
    1. Nadpisujemy `T` przez wynik XOR(`X`, `V[j]`)
    1. Nadpisujemy `X` scryptBlockMix(`T`)
1. Po wykonaniu powyższej pętli zwracamy ostatni `X`


### Algorytm `Scrypt`
Wracając do początku sekcji - algorytm można przedstawić w 3 krokach.
Zapis `PBKDF2-HMAC-SHA-256` oznacza, że używamy algorytmu `PBKDF2` (Password-Based Key Derivation Function - Opartej na haśle Funkcji Wyprowadzania Klucza) z `HMAC-SHA-256` jako `PRF` (Pseudorandom Function - Funkcja Pseudolosowa).

**Oficjalna reprezentacja algorytmu `scrypt`:**

```bash
Input:
        P       Passphrase, an octet string.
        S       Salt, an octet string.
        N       CPU/Memory cost parameter, must be larger than 1,
                a power of 2, and less than 2^(128 * r / 8).
        r       Block size parameter.
        p       Parallelization parameter, a positive integer
                less than or equal to ((2^32-1) * hLen) / MFLen
                where hLen is 32 and MFlen is 128 * r.
        dkLen   Intended output length in octets of the derived
                key; a positive integer less than or equal to
                (2^32 - 1) * hLen where hLen is 32.

Output:
        DK      Derived key, of length dkLen octets.

Steps:
    1.  Initialize an array B consisting of p blocks of 128 * r octets
        each:
        B[0] || B[1] || ... || B[p - 1] = PBKDF2-HMAC-SHA256 (P, S, 1, p * 128 * r)

    2.  for i = 0 to p - 1 do
        B[i] = scryptROMix (r, B[i], N)
    end for

    3.  DK = PBKDF2-HMAC-SHA256 (P, B[0] || B[1] || ... || B[p - 1], 1, dkLen)
```

Jako wejście do funkcji `scrypt` podajemy hasło (ciąg bajtów) `P`, sól (ciąg bajtów) `S`, parametr kosztu Procesora/Pamięci `N`, który musi być większy od 1, potęgą 2 i mniejszy od 2^(128 * r / 8), rozmiar bloku `r`, parametr Zrównoleglenia `P`, który musi być większy od zera, całkowity i mniejszy lub równy ((2^32-1) * 32) / (128 * `r`), porządaną długość klucza `dkLen`, która musi być większa od zera, całkowita  i mniejsza lub równa (2^32 - 1) * 32.

Kroki:
1. Inicjujemy tablicę `B` składającą się z `p` bloków o długości 128 * `r` bajtów każdy.
    - Wspomniane powyżej bloki otrzymujemy przez wywołanie funkcji PBKDF2-HMAC-SHA256(`P`, `S`, 1, `p` * 128 * `r`)
1. Wykonujemy `p` razy poniższe instrukcje:
    1. Do elementu `i`-tego tablicy `B` `B[i]`, gdzie `i` to indeks wykonywanej pętli przypisujemy wynik scryptROMix(`r`, `B[i]`, `N`)
1. Po wykonaniu powyższej pętli zwracamy wynik PBKDF2-HMAC-SHA256(`P`, `B`, 1, `dkLen`)

### Atak kanałem bocznym na `scrypt`

`scrypt` jest wrażliwy na `Atak Czasowy na Cache`.

`Atak czasowy na Cache` - ten typ ataku polega na pozyskiwaniu informacji przez badanie dostępności lub niedostępności danych w cache procesora.

Jedną z metod przeprowadzenia tego ataku jest metoda Yuval'a Yarom'a `PRIME+PROBE` i jest najlepszą metodą, której można użyć przeciwko scryptowi.

`PRIME+PROBE` (dosł. Zastaw i Sonduj) - Metoda Ataku polegająca na celowym wypchnięciu danych ofiary z cache procesora (Zastaw) przez uzyskanie dostępu do danych, które atakujący wie, że spowodują wypchnięcie z cache danych ofiary. Następnie, atakujący czeka pewien czas dając ofierze uzyskać dostęp do swoich danych, po czym próbuje uzyskać dostęp do swoich danych (Sonduj). Jeżeli czas dostępu jest relatywnie krótki, to oznacza, że ofiara nie uzyskiwała dostępu do swoich danych, bo dane atakującego wciaż są dostępne w cache. Jeżeli czas dostępu jest dłuższy, oznacza to, że ofiara użyskała dostęp do badanych danych, ponieważ dane atakującego zostały zepchnięte do pamięci niższego poziomu.
Atak ten może zostać zastosowany w dowolnym momencie kiedy atakujący współdzieli z ofiarą jakiś poziom cache procesora. To może się zdarzyć kiedy atakujący i ofiara są procesami na tej samej maszynie, ale również kiedy atakujący i ofiara pracują na różnych maszynach wirtualnych, hostowanych na jednej maszynie fizycznej. Wystarczy współdzielenie przez nich cache procesora.

---

**Procedura ataku na `scrypt`:**  
To co sprawia, że `Atak Czasowy na Cache` jest możliwy to podany poniżej kod w funkcji `scryptROMix`:
```bash
    j = Integerify(X) mod N
    T = X xor V[j]
    X = scryptBlockMix(T)
```

Jak zostało wspomniane wcześniej, tablica `V` wykorzystywana wewnątrz scryptBlockMix nadaje algorytmowi `scrypt` własność `Memory Hardness`. `V` przechowuje wszystkie powtarzające się hasze `B` takiego, że `V[n]` to rezultat haszowania `B` `n` razy. W powyższych linijkach elementy `V` wykorzystywane są w liczeniu rezultatu `scrypt` przez interpretowanie unikalnych haszy pochodzących od hasła podanego przez użytkownika do funkcji `PBKDF2-HMAC-SHA256` jako liczby całkowite użyte do indeksowania tablicy `V` (zmienna `j`). Ponieważ dostęp do tych danych zależy od hasła, obserwacja dostępu do pamięci pozwoli na przewidzenie rezultatu `PBKDF2-HMAC-SHA256` w pierwszym kroku algorytmu `scrypt`.

Zdobycie wystarczająco dużo informacji o haszu `PBKDF2-HMAC-SHA256` hasła ofiary pozwala atakującemu zredukować atak na `scrypt` do ataku na sam algorytm `PBKDF2-HMAC-SHA256`, w ten sposób całkowicie unikając mierzenia się z `Memory Hardness` algorytmu `scrypt`. Mówiąc precyzyjniej, w momencie kiedy atakujący zaobserwuje sygnaturę dostępu do pamięci `scrypt`, może zbudować słownik haszy `PBKDF2-HMAC-SHA256` i policzyć jakie są ich hasła i policzyć jakie będą ich sygnatury dostępu do pamięci i porównać je z sygnaturami haszy w słowniku. W ten sposób udaje się całkowicie ominąć `Memory Hardness` algorytmu `scrypt` przerzucając atak na algorytm `PBKDF2-HMAC-SHA256` i pozwalając atakującemu zastosować wszystkie skróty, żeby które uniemożliwić algorytm `scrypt` został zaprojektowany.

# Czym jest algorytm `yescrypt`?

`Yescrypt` jest funkcją wyprowadzania klucza opartą na haśle. Został stworzony na bazie `scrypt`.  
Yescrypt jest domyślną funkcją haszującą dla systemów Debian 11, Fedora 35+, Kali Linux 2021.1+ oraz Ubuntu 20.04+.

Zdaje się on być najlepiej skalującym algorytmem do haszowania. Oferuje prawie optymalne bezpieczeństwo ataków bruteforce kosztem złożoności.  
Warto mieć na uwadze, że w przypadku dużych implementacji będzie to stosunkowo mała część całkowitej autoryzacji. 

| Zalety | Wady |
| --- | --- |
| Lepsza ochrona przed atakami | Złożoność |
| Oparte na zaakceptowanych przez NIST funkcjach haszujących (HMAC, PBKDF2, SHA-256)| Podatne na atak kanałem bocznym (Cache-timing)|
| Czas działania można regulować w zależności od wykorzystania pamięci | Wspierany w mniejszej ilości technologi |

Yescrypt nie wygrał w [PHC](https://en.wikipedia.org/wiki/Password_Hashing_Competition). Zwycięzcą w 2015 roku został `Argon2`. 
Mimo to yescrypt dostał specjalne wyróżnienie (razem z `Catena`, `Lyra2`, `Makwa`).

Istnieje też `yespower`, który jest PoW (proof of work) zbudowanym na scrypcie. Zamiast wyprowadzać klucze i haszować hasła yespower jest przeznaczony do przetwarzania nagłówków bloków w `blockchain` (włączając wartość nounce).


# Algorytm `Balloon Hashing`

Kolejna odnoga funkcji KDF. Autorzy zapewniają, że:
1. Algorytm wykazuje właściwości `memory-hardness`. 
1. Jest bazowany na podstawowych funkcjach (SHA-512 ...). Zatem może być używany w dowolnym standardzie.
1. Odporny na ataki kanłem bocznym (side-channel attacks). Dostęp do pamięci jest niezależny od danych, które są haszowane.
1. Łatwy w implementacji a złożoność obliczeniowa jest podobna do innych algorytmów tego sortu.


# Podsumowanie

|Własność|scrypt|yescrypt|baloon hashing|
|---|---|---|---|
|Memory-Hardness|Tak|Tak|Tak|
|Bazowana na 'Standard Primitives' (np. SHA-512 itd.)|Tak|Tak|Tak|
|Odporność na ataki Cache (kanałem bocznym)|Nie<sup>1</sup>|Nie|Tak|
|Złożoność|Ω(n<sup>2</sup>ω)|Ω(n<sup>2</sup>ω)|Ω(n<sup>2</sup>/log n)<sup>2</sup>|


<sup>1</sup>Zostało udowodnione, że funkcja ROMix jest sekwencyjnie odporna dla wyroczni losowej. Bezpieczeństwo scryptu opiera się na założeniu, że BlockMix() wykorzystujący Salsa20/8 nie wykazuje, żadnych "skrótów" które pozwoliłyby iterować łatwiej niż w wyroczni losowej.  
<sup>2</sup>W ustawieniu równoległym Ω(n<sup>2</sup>). Dla rundy (odpowiednie warunki)  Ω(n<sup>5/3</sup>)<sup>2</sup>. Funkcja wykazująca własność memory-hard Ω(n<sup>2</sup>/log n).  


### Wnioski
1. Warto używać yescrypta gdy chcemy bronić się przed atakami równoległymi ale dostęp do pamięci będzie zależny od hasła. Jest to słabe podejście jeśli adwersach może mieć dostęp do takiej informacji.  
1. Baloon Hashing jest przydatny w obronie przed atakami sekwencyjnymi. Jest dostęp do pamięci jest niezależny od hasła. Jest asymptotycznie słabszy przy dużych atakach równoległych.
[Źródło](https://eprint.iacr.org/2016/027.pdf)

### Bibliografia

1. Funkcje wyprowadzania klucza (KDF) wykorzystujące odwzorowanie logistyczne - Grzegorz Frejek
1. https://qvault.io/cryptography/key-derivation-functions/
1. https://cryptobook.nakov.com/mac-and-key-derivation/hmac-and-key-derivation
1. https://cryptography.io/en/latest/hazmat/primitives/key-derivation-functions/
1. https://books.google.com/books?id=OAkhkLSxxxMC&pg=PA158
1. http://cacr.uwaterloo.ca/hac/
1. http://www.cs.ucdavis.edu/~rogaway/classes/227/spring05/book/main.pdf
1. https://books.google.com/books?id=UuNKmgv70lMC&pg=PA455
1. https://books.google.com/books?id=nErZY4vYHIoC&pg=PA321
1. https://web.archive.org/web/20070605092733/http://netlab.cs.ucla.edu/wiki/files/shannon1949.pdf
1. http://events.iaik.tugraz.at/RFIDSec08/Papers/Publication/04%20-%20ONeill%20-%20Low%20Cost%20SHA-1%20-%20Slides.pdf
1. https://web.archive.org/web/20030322053727/http://cm.bell-labs.com/cm/cs/who/dmr/passwd.ps
1. http://www.bsdcan.org/2009/schedule/events/147.en.html
1. http://www.akkadia.org/drepper/SHA-crypt.txt
1. https://docs.python.org/3/library/hmac.html
1. https://www.apprendre-en-ligne.net/crypto/bibliotheque/feistel/index.html
1. https://eprint.iacr.org/2011/565
1. https://crypto.stanford.edu/cs359c/17sp/projects/MarkAnderson.pdf
1. https://www.openwall.com/yescrypt/
1. https://www.tarsnap.com/scrypt.html
1. https://qvault.io/cryptography/very-basic-intro-to-the-scrypt-hash/
1. https://courses.csail.mit.edu/6.857/2016/files/salsa20.py
1. https://github.com/Daeinar/salsa20
1. https://crypto.stanford.edu/balloon/
1. https://nvlpubs.nist.gov/nistpubs/SpecialPublications/NIST.SP.800-63b.pdf
1. http://www.tarsnap.com/scrypt/scrypt.pdf
1. https://eprint.iacr.org/2016/027.pdf
1. https://cr.yp.to/snuffle/spec.pdf
1. https://datatracker.ietf.org/doc/html/rfc7914
1. https://securitycurrent.com/key-management-a-fast-growing-space/